In [1]:
%defaultDatasource jdbc:h2:mem:db

# Criação das tabelas Country e CasosCovid

In [6]:
CREATE TABLE Country (
    iso_code VARCHAR(8) NOT NULL,
    name VARCHAR(50),
    continent VARCHAR(50),
    population DOUBLE,
    population_density DOUBLE,
    median_age DOUBLE,
    aged_65_older DOUBLE,
    aged_70_older DOUBLE,
    gdp_per_capita DOUBLE,
    extreme_poverty DOUBLE,
    cardiovasc_death_rate DOUBLE,
    diabetes_prevalence DOUBLE,
    female_smokers DOUBLE,
    male_smokers DOUBLE,
    handwashing_facilities DOUBLE,
    hospital_beds_per_thousand DOUBLE,
    life_expectancy DOUBLE,
    human_development_index DOUBLE,
    PRIMARY KEY (iso_code)
) AS SELECT
    iso_code,
    location,
    continent,
    population,
    population_density,
    median_age,
    aged_65_older,
    aged_70_older,
    gdp_per_capita,
    extreme_poverty,
    cardiovasc_death_rate,
    diabetes_prevalence,
    female_smokers,
    male_smokers,
    handwashing_facilities,
    hospital_beds_per_thousand,
    life_expectancy,
    human_development_index
    FROM CSVREAD('owid-country-data.csv');

CREATE TABLE CasosCovid (
    iso_code VARCHAR(8) NOT NULL,
    date DATE NOT NULL,
    total_cases DOUBLE,
    new_cases DOUBLE, 
    total_deaths DOUBLE,
    new_deaths DOUBLE,
    total_tests DOUBLE,
    FOREIGN KEY (iso_code) REFERENCES Country (iso_code)
) AS SELECT
    iso_code,
    date,
    total_cases,
    new_cases,
    total_deaths,
    new_deaths,
    total_tests
    FROM CSVREAD('owid-covid-data.csv')

In [27]:
SELECT * FROM Country;
SELECT * FROM CasosCovid;

# Algumas queries úteis para analise rápida

## 1) Ultimos números de mortes em Taiwan

In [23]:
SELECT c.name, co.total_deaths, co.date
    FROM Country c, CasosCovid co
    WHERE c.iso_code = co.iso_code and c.iso_code = 'TWN'
    ORDER BY date DESC
    LIMIT 1
    

## 2) País que contém mais fumantes

In [26]:
SELECT name, female_smokers + male_smokers as Fumantes
    FROM Country
    ORDER BY Fumantes

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [ ]:
SELECT CodMedicamento, Count(*) AS Quantidade
    FROM UsoMedicamento
    GROUP BY CodMedicamento
    ORDER BY Quantidade DESC
    LIMIT 1;

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [9]:
SELECT NomeMedicamento, COUNT(*) QtdadeUsados
    FROM Medicamento, UsoMedicamento
    WHERE Codigo = CodMedicamento
    GROUP BY CodMedicamento
    ORDER BY QtdadeUsados DESC
    LIMIT 1;

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [10]:
SELECT AVG(QtdMedicamentos) Media
    FROM (SELECT COUNT(*) QtdMedicamentos
              FROM UsoMedicamento
              GROUP BY IdPessoa);

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [11]:
SELECT IdPessoa, COUNT(*) QtdMedicamentos
    FROM UsoMedicamento
    GROUP BY IdPessoa
    HAVING QtdMedicamentos > (SELECT AVG(QtdMedicamentos) Media
                                    FROM (SELECT COUNT(*) QtdMedicamentos
                                            FROM UsoMedicamento
                                            GROUP BY IdPessoa))

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [12]:
SELECT IdadePessoa, AVG(QtdMedicamentos) Media
    FROM (SELECT Idade IdadePessoa, COUNT(*) QtdMedicamentos
              FROM Pessoa, UsoMedicamento
              WHERE Id = IdPessoa
              GROUP BY IdPessoa)
    GROUP BY IdadePessoa
    ORDER BY IdadePessoa;

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [ ]:
CREATE VIEW UsoMedicamentoB AS
    SELECT IdPessoa IdPessoaB, CodMedicamento CodMedicamentoB
        FROM UsoMedicamento;

CREATE VIEW MedicamentosConjunto AS
    SELECT U1.CodMedicamento medicamentoA, U2.CodMedicamentoB medicamentoB
        FROM UsoMedicamento U1, UsoMedicamentoB U2
        WHERE U1.IdPessoa = U2.IdPessoaB AND U1.CodMedicamento > U2.CodMedicamentoB;

SELECT * FROM MedicamentosConjunto;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [ ]:
SELECT medicamentoA, medicamentoB, COUNT(*) QuantidadeConjunta
    FROM MedicamentosConjunto
    GROUP BY medicamentoA, medicamentoB;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [3]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [5]:
UPDATE Matriz
    SET IdadeAte60 = TRUE
    WHERE Idade <=60;

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [6]:
UPDATE Matriz
    SET MaisUmAnoMedicamento = TRUE
    WHERE Id IN (SELECT DISTINCT IdPessoa
                     FROM UsoMedicamento
                     WHERE DiasUso > 365);

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [7]:
UPDATE Matriz
    SET MedicamentosAcimaMedia = TRUE
    WHERE Id IN (SELECT IdPessoa
                    FROM UsoMedicamento
                    GROUP BY IdPessoa
                    HAVING COUNT(*) > (SELECT AVG(QtdMedicamentos) Media
                                            FROM (SELECT COUNT(*) QtdMedicamentos
                                                      FROM UsoMedicamento
                                                      GROUP BY IdPessoa)));
              
SELECT * FROM Matriz WHERE MedicamentosAcimaMedia = TRUE;

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

A análise proposta é descobrir se o número de idosos que utilizam medicamentos a mais de ano é maior que pessoas com menos de 60 anos  

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [31]:
CREATE VIEW Menos60 AS 
SELECT COUNT (*) QtdadeMenos60
FROM Matriz
WHERE IdadeAte60 = FALSE and MaisUmAnoMedicamento = TRUE;

CREATE VIEW Mais60 AS 
SELECT COUNT (*) QtdadeMais60
FROM Matriz
WHERE IdadeAte60 = TRUE and MaisUmAnoMedicamento = TRUE;

SELECT QtdadeMenos60, QtdadeMais60
FROM Menos60, Mais60